In [1]:
import pandas as pd

csv_path="data/PI1070.csv"
df=pd.read_csv(csv_path)

In [2]:
df

,monomer_ID,smiles,mol_weight_monomer,atomic_weight_mean,vdw_volume_monomer,qm_total_energy_monomer,qm_homo_monomer,qm_lumo_monomer,qm_dipole_monomer,qm_dipole_x_monomer,...,TC_improper_min,TC_improper_max,TC_improper_std,TC_improper_count,TC_kspace,TC_kspace_min,TC_kspace_max,TC_kspace_std,TC_kspace_count,polymer_class
0,PI1,*CC*,28.054,4.675667,45.008,-2.096767e+05,-12.037272,3.128817,0.000009,0.000006,...,0.000000,0.000000,0.000000,5.0,0.000014,-0.000028,0.000029,0.000024,5.0,1
1,PI2,*C(C*)C,42.081,4.675667,61.624,-3.129489e+05,-11.576865,3.039961,0.077278,0.003980,...,0.000000,0.000000,0.000000,5.0,0.000279,-0.000165,0.000562,0.000302,5.0,1
2,PI3,*C(C*)CC,56.108,4.675667,78.424,-4.162192e+05,-11.423908,2.987135,0.081852,-0.003192,...,0.000000,0.000000,0.000000,5.0,0.000059,-0.000043,0.000288,0.000133,5.0,1
3,PI4,*C(C*)CCC,70.135,4.675667,95.392,-5.194934e+05,-11.227201,3.052917,0.075347,0.003630,...,0.000000,0.000000,0.000000,5.0,0.000081,0.000031,0.000146,0.000046,5.0,1
4,PI5,*C(C*)C(C)C,70.135,4.675667,95.216,-5.194974e+05,-11.171034,2.957730,0.085636,0.032327,...,0.000000,0.000000,0.000000,5.0,0.000057,-0.000576,0.000607,0.000449,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,PI1134,*c1ccc(C(c2ccc(-c3nc4cc(-c5ccc6nc(*)[nH]c6c5)c...,534.463,9.717509,427.216,-5.080617e+06,-7.934564,0.253573,1.613474,-1.594711,...,0.000338,0.000602,0.000118,4.0,0.000490,-0.000855,0.001466,0.001136,4.0,16
1073,PI1135,*c1ccc(C(c2ccc(-c3nc4ccc(-c5ccc6nc(*)[nH]c6c5)...,534.463,9.717509,426.976,-5.080617e+06,-7.884157,0.224698,3.065224,0.632658,...,0.000487,0.000969,0.000242,3.0,0.001890,0.000755,0.003050,0.001148,3.0,16
1074,PI1136,*c1cccc(S(=O)(=O)NCCCCCCNS(*)(=O)=O)c1,318.420,8.379474,277.176,-4.401789e+06,-9.913705,0.724424,4.368539,-2.381232,...,0.000045,0.000172,0.000052,5.0,0.002583,0.002387,0.002991,0.000244,5.0,20
1075,PI1137,*c1ccc(S(*)(=O)=O)cc1,140.163,10.781769,117.544,-2.050694e+06,-10.235109,0.492464,5.217217,-4.925100,...,-0.000005,0.000294,0.000115,5.0,0.001335,-0.000409,0.004160,0.001702,5.0,20


In [21]:
smiles_column="smiles"
property_column_dict={
    "density":["density",1,"g/cm3"],
    "Cp":["heat capacity at constant pressure",1,"J/Kg/K"],
    "Cv":["heat capacity at constant volume",1,"J/Kg/K"],
    "refractive_index":["refractive index",1,"-"],
    "thermal_conductivity":["thermal conductivity",1,"W/m/K"],
    "bulk_modulus":["bulk modulus",10**-9,"GPa"],
    "compressibility":["compressibility",10**9,"1/GPa"],
    "thermal_diffusivity":["thermal diffusivity",10**6,"mm^2/s"],
    "Rg":["Radius of gyration",1,"angstrom"],
    "qm_dipole_monomer":["monomer's dipole moment",1,"Debye"],
    "vdw_volume_monomer":["monomer's van der Waals volume",1,"angstrom^3"],
}

In [38]:
fact_list=[]

target_prop_key="density"

for prop_key in property_column_dict.keys():
    prop_name,prop_scale,prop_unit=property_column_dict[prop_key]
    prop_df=df[[smiles_column,target_prop_key]].dropna()

    for i in range(len(prop_df)):
        smiles=prop_df.iloc[i][smiles_column]
        prop=prop_df.iloc[i][target_prop_key]
        val=prop*prop_scale
        #小数点3桁まで
        val=round(val,3)
        fact={"SMILES":smiles,"Property":f"simulated {prop_name} by RadonPy","Value":val,"unit":prop_unit,
              "CompName":"","Source":"RadonPy",
              }
        fact_list.append(fact)

In [39]:
fact_list[5000]

{'SMILES': '*c1cccc(Oc2ccc(S(=O)(=O)c3ccc(Oc4cccc(N5C(=O)c6ccc(-c7ccc8c(c7)C(=O)N(*)C8=O)cc6C5=O)c4)cc3)cc2)c1',
 'Property': 'simulated thermal conductivity by RadonPy',
 'Value': 1.272,
 'unit': 'W/m/K',
 'CompName': '',
 'Source': 'RadonPy'}

In [40]:
import datasets
dataset=datasets.Dataset.from_list(fact_list)

In [41]:
dataset

Dataset({
    features: ['SMILES', 'Property', 'Value', 'unit', 'CompName', 'Source'],
    num_rows: 11847
})

In [42]:

dataset.push_to_hub("kanhatakeyama/material-properties",split="RadonPy")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kanhatakeyama/material-properties/commit/47925a749eec80291a7be0df6a64546e592f08f3', commit_message='Upload dataset', commit_description='', oid='47925a749eec80291a7be0df6a64546e592f08f3', pr_url=None, pr_revision=None, pr_num=None)